In [1]:
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [65]:
def pickleIt(csvfile):
    df = pd.read_csv("data/%s" % csvfile)
    df.to_pickle("data/%s.pickle" % csvfile)

In [2]:
df = pd.read_csv("data/games.csv")

#labbr = list(set(list(df.homeTeamAbbr) + list(df.visitorTeamAbbr)))
#labbr.sort()
#dabbr = {labbr[i]:i+1 for i in range(len(labbr))}

#df['ihomeTeamAbbr'] = df['homeTeamAbbr'].apply(lambda x: dabbr[x])
#df['ivisitorTeamAbbr'] = df['visitorTeamAbbr'].apply(lambda x: dabbr[x])

df['homeTeamAbbr'] = df['homeTeamAbbr'].astype('category')
df['visitorTeamAbbr'] = df['visitorTeamAbbr'].astype('category')

df.to_pickle("data/games.csv.x.pickle")

In [3]:
#pickleIt("players.csv")

def fheight(h):
    try:
        return int(h)
    except:
        a = h.split('-')
        return int(a[0])*12+int(a[1])

def shortname(s):
    a = s.split()
    return "%s.%s" % (a[0][0], a[-1])

df = pd.read_csv("data/players.csv")
df['shortName'] = df['displayName'].apply(shortname).astype('category')
df['position'] = df['position'].astype('category')
df['height'] = df['height'].apply(fheight)

df.to_pickle("data/players.csv.x.pickle")

In [4]:
import re

#pickleIt("plays.csv")
df_players = pd.read_pickle("data/players.csv.x.pickle")
df = pd.read_csv("data/plays.csv")

re1 = re.compile(r'(to|for) (\S+)')

re2a = re.compile(r'INTERCEPTED by (\S+)')
re2b = re.compile(r'\s\((\S+)\)')


def reOPlayer(s):
    try:
        snm = re1.search(s).group(2).strip(".")
        p = df_players.query('shortName=="%s"' % snm)
        return p.values[0][0]
    except:
        return 0
    
def reDPlayer(s):
    try:
        r = re2a.search(s)
        if r is None: r = re2b.search(s)
        snm = r.group(1).strip(".")
        p = df_players.query('shortName=="%s"' % snm)
        return p.values[0][0]
    except:
        return 0

df['oid'] = df['playDescription'].apply(reOPlayer)
df['did'] = df['playDescription'].apply(reDPlayer)


In [5]:
df['playType'] = df['playType'].astype('category')
df['offenseFormation'] = df['offenseFormation'].apply(str).astype('category')
df['typeDropback'] = df['typeDropback'].apply(str).astype('category')
df['passResult'] = df['passResult'].apply(str).astype('category')
df['yardlineSide'] = df['yardlineSide'].apply(str).astype('category')
df['absoluteYardlineNumber'] = df['absoluteYardlineNumber'].fillna(0)
        
df.to_pickle("data/plays.csv.x.pickle")

In [6]:
DPOS = ['CB', 'LB', 'OLB', 'ILB', 'MLB', 'FS', 'S', 'SS', 'LS', 'DB', 'DE', 'DT', 'DL']
for i in range(1,18):
    csvfile = "data/week%d.csv" % i
    df = pd.read_csv(csvfile)
    df['event'] = df['event'].astype('category')
    df['upos'] = df['position'] + df.groupby(by=['gameId','playId','frameId','position']).cumcount().astype(str)
    df['position'] = df['position'].apply(str).astype('category')
    df['iteam'] = df['team'].apply(lambda x : 1 if x == 'home' else 0)
    df['iplayDirection'] = df['playDirection'].apply(lambda x : -1 if x == 'left' else 1)
    df['route'] = df['route'].apply(str).astype('category')
    df['time'] = pd.to_datetime(df['time'])
    df['one'] = 1
    df['bdefense'] = df['position'].apply(lambda x: -1 if x in DPOS else 1)
    df['nflId'] = df['nflId'].apply(lambda id: -1 if math.isnan(id) else id )
    df['nflId'] = pd.to_numeric(df['nflId'], downcast='integer')
    df.to_pickle("%s.x.pickle" % csvfile)


In [8]:
df_plays =  pd.read_pickle("data/plays.csv.x.pickle").set_index(['gameId', 'playId'])
df_games = pd.read_pickle("data/games.csv.x.pickle").set_index(['gameId'])

df_pg = df_plays.join(df_games)
df_pg['los'] = df_pg.apply(lambda d : 100.-d['yardlineNumber'] if d['possessionTeam']==d['yardlineSide'] else d['yardlineNumber'], axis=1).fillna(-1)
df_pg['defTeam'] = df_pg.apply(lambda d : d['homeTeamAbbr'] if d['possessionTeam'] != d['homeTeamAbbr'] else d['visitorTeamAbbr'] , axis=1 )
df_pg['defTeam'] = df_pg['defTeam'].astype('category')

for i in range(1,18):
    file = "data/week%d.csv.x.pickle" % i
    print(file)
    df = pd.read_pickle(file).set_index(['gameId','playId','frameId','nflId'])
    df = df[~df.index.duplicated(keep='first')]
    df = df.join(df_pg[['los','yardsToGo','absoluteYardlineNumber', 'down','passResult','oid','did', 'defTeam',\
            'offenseFormation','defendersInTheBox', 'numberOfPassRushers','typeDropback']])
    
    df = df.reset_index()
    df['nflId'] = df.apply(lambda d : -2 if d['nflId']==d['oid'] else d['nflId'], axis=1 )
    df = df.set_index(['gameId','playId','frameId','nflId'])
    
    df1 = df.query('nflId==-1')
    df['fx'] = df.loc[:,:,:,:].x - df1.loc[:,:,:,-1].x
    df['fy'] = df.loc[:,:,:,:].y - df1.loc[:,:,:,-1].y
    df['d2fb'] = (df['fx']**2 + df['fy']**2).apply(math.sqrt)
    
    df2 = df.query('nflId==-2')
    df['ox'] = df.loc[:,:,:,:].x - df2.loc[:,:,:,-2].x
    df['oy'] = df.loc[:,:,:,:].y - df2.loc[:,:,:,-2].y
    df['d2oid'] = (df['ox']**2 + df['oy']**2).apply(math.sqrt)
    
    df = df.reset_index()
    df.to_pickle("data/week%d.csv.y.pickle" % i)
    
    

data/week1.csv.x.pickle
data/week2.csv.x.pickle
data/week3.csv.x.pickle
data/week4.csv.x.pickle
data/week5.csv.x.pickle
data/week6.csv.x.pickle
data/week7.csv.x.pickle
data/week8.csv.x.pickle
data/week9.csv.x.pickle
data/week10.csv.x.pickle
data/week11.csv.x.pickle
data/week12.csv.x.pickle
data/week13.csv.x.pickle
data/week14.csv.x.pickle
data/week15.csv.x.pickle
data/week16.csv.x.pickle
data/week17.csv.x.pickle
